<a href="https://colab.research.google.com/github/JesseJames50/Analise-Dados-Financeiros/blob/main/Candle_Pavio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install yfinance
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [27]:
import yfinance as yf
import pandas as pd
import numpy as np

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR3.SA", start="2021-01-01")

# Calcular o corpo e a amplitude do candle
data['Body'] = abs(data['Close'] - data['Open'])
data['Range'] = data['High'] - data['Low']
data['Candle'] = data['Body'] / data['Range'] <= 0.2  # Candles com corpo até 20% da amplitude

# Identificar gap de abertura após o candle de pavio
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['EntrySignal'] = (data['Candle'] & (data['Gap'] >= 0.005)).shift(1)  # Gap de pelo menos 0.5%
data['EntrySignal'] = data['EntrySignal'].fillna(False)  # Trata NaN como False

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir resultados
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed

            EntrySignal  EntryPrice  ExitPrice  Returns  CumulativeReturns
Date                                                                      
2021-01-04        False   29.459999  30.400000      0.0             0.0000
2021-01-05        False   30.500000  30.750000      0.0             0.0000
2021-01-06        False   30.920000  31.650000      0.0             0.0000
2021-01-07        False   31.879999  31.590000      0.0             0.0000
2021-01-08        False   31.120001  31.290001      0.0             0.0000
...                 ...         ...        ...      ...                ...
2024-05-03        False   42.020000  42.220001      0.0             0.1316
2024-05-06        False   42.310001  43.240002      0.0             0.1316
2024-05-07        False   42.910000  43.700001      0.0             0.1316
2024-05-08        False   43.500000  44.470001      0.0             0.1316
2024-05-09        False   44.500000  44.130001      0.0             0.1316

[834 rows x 5 columns]
T

In [26]:
# Adicionando médias móveis como sinal de entrada
import yfinance as yf
import pandas as pd
import numpy as np

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR4.SA", start="2021-01-01")

# Calcular o corpo e a amplitude do candle
data['Body'] = abs(data['Close'] - data['Open'])
data['Range'] = data['High'] - data['Low']
data['Candle'] = data['Body'] / data['Range'] <= 0.2  # Candles com corpo até 20% da amplitude

# Identificar gap de abertura após o candle de pavio
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['EntrySignal'] = (data['Candle'] & (data['Gap'] >= 0.005)).shift(1)  # Gap de pelo menos 0.5%

# Adicionando Médias Móveis Simples
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# Filtrar sinais com base na direção das médias móveis
data['Trend'] = data['SMA50'] > data['SMA200']
data['EntrySignal'] = data['EntrySignal'] & data['Trend']

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir resultados
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed

            EntrySignal  EntryPrice  ExitPrice  Returns  CumulativeReturns
Date                                                                      
2021-01-04        False   28.900000  30.040001      0.0           0.000000
2021-01-05        False   30.160000  30.100000      0.0           0.000000
2021-01-06        False   30.340000  31.000000      0.0           0.000000
2021-01-07        False   31.459999  31.120001      0.0           0.000000
2021-01-08        False   30.610001  30.860001      0.0           0.000000
...                 ...         ...        ...      ...                ...
2024-05-03        False   39.869999  40.160000      0.0           0.008556
2024-05-06        False   40.150002  40.650002      0.0           0.008556
2024-05-07        False   40.349998  41.270000      0.0           0.008556
2024-05-08        False   41.060001  41.669998      0.0           0.008556
2024-05-09        False   41.840000  41.580002      0.0           0.008556

[834 rows x 5 columns]
T

In [25]:
# Adicionando RSI como sinal de entrada
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR4.SA", start="2021-01-01")

# Calcular o corpo e a amplitude do candle
data['Body'] = abs(data['Close'] - data['Open'])
data['Range'] = data['High'] - data['Low']
data['Candle'] = data['Body'] / data['Range'] <= 0.20  # Candles com corpo até 20% da amplitude

# Identificar gap de abertura após o candle de pavio
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['EntrySignal'] = (data['Candle'] & (data['Gap'] >= 0.005)).shift(1)  # Gap de pelo menos 0.5%

# Adicionando RSI
data['RSI'] = calculate_rsi(data)
data['RSI_Entry'] = (data['RSI'] > 30) & (data['RSI'] < 70)

# Filtrar sinais com base no RSI
data['EntrySignal'] = data['EntrySignal'] & data['RSI_Entry']

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir resultados
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed

            EntrySignal  EntryPrice  ExitPrice  Returns  CumulativeReturns
Date                                                                      
2021-01-04        False   28.900000  30.040001      0.0           0.000000
2021-01-05        False   30.160000  30.100000      0.0           0.000000
2021-01-06        False   30.340000  31.000000      0.0           0.000000
2021-01-07        False   31.459999  31.120001      0.0           0.000000
2021-01-08        False   30.610001  30.860001      0.0           0.000000
...                 ...         ...        ...      ...                ...
2024-05-03        False   39.869999  40.160000      0.0          -0.005622
2024-05-06        False   40.150002  40.650002      0.0          -0.005622
2024-05-07        False   40.349998  41.270000      0.0          -0.005622
2024-05-08        False   41.060001  41.669998      0.0          -0.005622
2024-05-09        False   41.840000  41.580002      0.0          -0.005622

[834 rows x 5 columns]
T

In [30]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carregar dados
data = yf.download("PETR4.SA", start="2017-01-01", end="2023-01-01")

# Preparar os dados
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['Target'] = data['Close'].shift(-20) > data['Close']  # Comparando o preço de hoje com o preço daqui a um mês

# Limpar dados NaN
data = data.dropna()

# Criar features e target
X = data[['SMA50']]
y = data['Target']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Usar o modelo para filtrar sinais de entrada
data['Predicted_Trend'] = model.predict(data[['SMA50']])
data['EntrySignal'] = data['Predicted_Trend'] & (data['Gap'] >= 0.005).shift(1)

# Análise de trades com base no novo sinal
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir resultados finais
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())


[*********************100%%**********************]  1 of 1 completed

Accuracy: 0.6228373702422145



<ipython-input-30-1171d6ce9174>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Predicted_Trend'] = model.predict(data[['SMA50']])


KeyError: 'Gap'

In [33]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Função para calcular o RSI - não usada diretamente aqui, mas útil se você quiser incluir análises adicionais
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR4.SA", start="2022-01-01")

# Preparar os dados para o modelo de machine learning
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1  # Correção para garantir a existência da coluna 'Gap'
data['Target'] = data['Close'].shift(-20) > data['Close']  # Comparando o preço de hoje com o preço daqui a um mês

# Limpar dados NaN para garantir integridade nos dados
data.dropna(inplace=True)

# Criar features e target para o modelo
X = data[['SMA50']]
y = data['Target']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de classificação
model = LogisticRegression()
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Usar o modelo para prever a tendência e filtrar sinais de entrada
data['Predicted_Trend'] = model.predict(data[['SMA50']])
data.loc[:, 'EntrySignal'] = data['Predicted_Trend'] & (data['Gap'] >= 0.005).shift(1)

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir os resultados finais
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed

Accuracy: 0.5277777777777778
            EntrySignal  EntryPrice  ExitPrice   Returns  CumulativeReturns
Date                                                                       
2022-03-15        False   31.500000  30.830000  0.000000           0.000000
2022-03-16         True   31.250000  30.010000 -0.039680          -0.039680
2022-03-17         True   30.090000  30.610001  0.017282          -0.023084
2022-03-18        False   30.910000  31.760000  0.000000          -0.023084
2022-03-21         True   31.990000  31.620001 -0.011566          -0.034383
...                 ...         ...        ...       ...                ...
2024-05-02        False   40.689999  39.889999  0.000000           0.055123
2024-05-03        False   39.869999  40.160000  0.000000           0.055123
2024-05-06        False   40.150002  40.650002  0.000000           0.055123
2024-05-07        False   40.349998  41.270000  0.000000           0.055123
2024-05-08        False   41.060001  41.669998  0.000000   

In [36]:
# integrar o RSI (Relative Strength Index) ao modelo de Random Forest para melhorar a análise
# das entradas de compra
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Função para calcular o RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR4.SA", start="2020-01-01")

# Preparar os dados para o modelo de machine learning
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['Target'] = data['Close'].shift(-20) > data['Close']

# Limpar dados NaN
data.dropna(inplace=True)

# Criar features e target
X = data[['SMA50']]
y = data['Target']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de classificação com Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Usar o modelo para prever a tendência e filtrar sinais de entrada
data['Predicted_Trend'] = model.predict(data[['SMA50']])
data.loc[:, 'EntrySignal'] = data['Predicted_Trend'] & (data['Gap'] >= 0.005).shift(1)

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir os resultados finais
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed


Accuracy: 0.5700483091787439
            EntrySignal  EntryPrice  ExitPrice  Returns  CumulativeReturns
Date                                                                      
2020-03-13        False   12.910000  13.090000      0.0           0.000000
2020-03-16        False   13.600000  13.000000      0.0           0.000000
2020-03-17        False   11.790000  11.290000      0.0           0.000000
2020-03-18        False   11.070000  12.210000      0.0           0.000000
2020-03-19        False   13.110000  12.000000      0.0           0.000000
...                 ...         ...        ...      ...                ...
2024-05-02        False   40.689999  39.889999      0.0           0.189851
2024-05-03        False   39.869999  40.160000      0.0           0.189851
2024-05-06        False   40.150002  40.650002      0.0           0.189851
2024-05-07        False   40.349998  41.270000      0.0           0.189851
2024-05-08        False   41.060001  41.669998      0.0           0.189

In [37]:
# RSI e sua utilização como uma das features para o modelo de Random Forest
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Função para calcular o RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR4.SA", start="2022-01-01")

# Preparar os dados para o modelo de machine learning
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['RSI'] = calculate_rsi(data)  # Calcular o RSI e adicioná-lo ao DataFrame
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['Target'] = data['Close'].shift(-20) > data['Close']

# Limpar dados NaN
data.dropna(inplace=True)

# Criar features e target para o modelo
X = data[['SMA50', 'RSI']]  # Adicionando RSI como uma feature
y = data['Target']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de classificação com Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Usar o modelo para prever a tendência e filtrar sinais de entrada
data['Predicted_Trend'] = model.predict(data[['SMA50', 'RSI']])
data.loc[:, 'EntrySignal'] = data['Predicted_Trend'] & (data['Gap'] >= 0.005).shift(1)

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir os resultados finais
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed


Accuracy: 0.7685185185185185
            EntrySignal  EntryPrice  ExitPrice  Returns  CumulativeReturns
Date                                                                      
2022-03-15        False   31.500000  30.830000  0.00000           0.000000
2022-03-16         True   31.250000  30.010000 -0.03968          -0.039680
2022-03-17        False   30.090000  30.610001  0.00000          -0.039680
2022-03-18        False   30.910000  31.760000  0.00000          -0.039680
2022-03-21        False   31.990000  31.620001  0.00000          -0.039680
...                 ...         ...        ...      ...                ...
2024-05-02        False   40.689999  39.889999  0.00000           0.048084
2024-05-03        False   39.869999  40.160000  0.00000           0.048084
2024-05-06        False   40.150002  40.650002  0.00000           0.048084
2024-05-07        False   40.349998  41.270000  0.00000           0.048084
2024-05-08        False   41.060001  41.669998  0.00000           0.048

In [40]:
#  MACD e sua utilização como uma das features para o modelo de Random Forest
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Função para calcular o MACD
def calculate_macd(data, slow=26, fast=12, signal=9):
    data['EMA_fast'] = data['Close'].ewm(span=fast, adjust=False).mean()
    data['EMA_slow'] = data['Close'].ewm(span=slow, adjust=False).mean()
    data['MACD'] = data['EMA_fast'] - data['EMA_slow']
    data['Signal'] = data['MACD'].ewm(span=signal, adjust=False).mean()
    data['MACD_Histogram'] = data['MACD'] - data['Signal']
    return data

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("PETR4.SA", start="2020-01-01")

# Preparar os dados para o modelo de machine learning
calculate_macd(data)
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['Target'] = data['Close'].shift(-20) > data['Close']

# Limpar dados NaN
data.dropna(inplace=True)

# Criar features e target para o modelo
X = data[['SMA50', 'MACD', 'Signal', 'MACD_Histogram']]
y = data['Target']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de classificação com Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Usar o modelo para prever a tendência e filtrar sinais de entrada
data['Predicted_Trend'] = model.predict(data[['SMA50', 'MACD', 'Signal', 'MACD_Histogram']])
data.loc[:, 'EntrySignal'] = data['Predicted_Trend'] & (data['Gap'] >= 0.005).shift(1)

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)
data['Returns'] = np.where(data['EntrySignal'], (data['ExitPrice'] / data['EntryPrice']) - 1, 0)
data['Returns'] = data['Returns'].fillna(0)
data['CumulativeReturns'] = (1 + data['Returns']).cumprod() - 1

# Exibir os resultados finais
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Returns', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Returns'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed


Accuracy: 0.8840579710144928
            EntrySignal  EntryPrice  ExitPrice   Returns  CumulativeReturns
Date                                                                       
2020-03-13        False   12.910000  13.090000  0.000000           0.000000
2020-03-16        False   13.600000  13.000000  0.000000           0.000000
2020-03-17         True   11.790000  11.290000 -0.042409          -0.042409
2020-03-18        False   11.070000  12.210000  0.000000          -0.042409
2020-03-19        False   13.110000  12.000000  0.000000          -0.042409
...                 ...         ...        ...       ...                ...
2024-05-02        False   40.689999  39.889999  0.000000           0.041552
2024-05-03        False   39.869999  40.160000  0.000000           0.041552
2024-05-06        False   40.150002  40.650002  0.000000           0.041552
2024-05-07        False   40.349998  41.270000  0.000000           0.041552
2024-05-08        False   41.060001  41.669998  0.000000   

In [44]:
# RSI (Relative Strength Index) e stop-loss no modelo de Random Forest para melhorar a análise
# das entradas de compra
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Função para calcular o RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Carregar dados de PETR4.SA usando yfinance
data = yf.download("wege3.SA", start="2020-01-01")

# Preparar os dados para o modelo de machine learning
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['Gap'] = data['Open'].shift(-1) / data['Close'] - 1
data['Target'] = data['Close'].shift(-20) > data['Close']

# Limpar dados NaN
data.dropna(inplace=True)

# Criar features e target
X = data[['SMA50']]
y = data['Target']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de classificação com Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Usar o modelo para prever a tendência e filtrar sinais de entrada
data['Predicted_Trend'] = model.predict(data[['SMA50']])
data.loc[:, 'EntrySignal'] = data['Predicted_Trend'] & (data['Gap'] >= 0.005).shift(1)

# Definir os preços de entrada e saída
data['EntryPrice'] = data['Open'].shift(-1)
data['ExitPrice'] = data['Close'].shift(-1)

# Calcular retorno potencial e aplicar stop-loss
stop_loss_threshold = -0.02  # -2%
data['Potential_Return'] = (data['ExitPrice'] / data['EntryPrice']) - 1
data['Stop_Loss'] = data['Potential_Return'] < stop_loss_threshold
data['Actual_Return'] = np.where(data['Stop_Loss'], stop_loss_threshold, data['Potential_Return'])

# Atualizar o cálculo de retornos cumulativos considerando o stop-loss
data['CumulativeReturns'] = (1 + data['Actual_Return']).cumprod() - 1

# Exibir os resultados finais
print(data[['EntrySignal', 'EntryPrice', 'ExitPrice', 'Actual_Return', 'CumulativeReturns']].dropna())

# Análise de desempenho
total_trades = data['EntrySignal'].sum()
average_return = data['Actual_Return'][data['EntrySignal']].mean()
cumulative_return = data['CumulativeReturns'].iloc[-1]

print(f"Total Trades: {total_trades}")
print(f"Average Return per Trade: {average_return:.2%}")
print(f"Cumulative Return: {cumulative_return:.2%}")


[*********************100%%**********************]  1 of 1 completed


Accuracy: 0.5748792270531401
            EntrySignal  EntryPrice  ExitPrice  Actual_Return  \
Date                                                            
2020-03-13        False   15.840000  13.705000      -0.040000   
2020-03-16        False   14.255000  14.340000       0.005963   
2020-03-17         True   12.950000  13.235000       0.022008   
2020-03-18        False   13.115000  14.365000       0.095311   
2020-03-19        False   15.575000  16.184999       0.039165   
...                 ...         ...        ...            ...   
2024-05-02        False   38.990002  38.770000      -0.005643   
2024-05-03        False   38.610001  38.630001       0.000518   
2024-05-06        False   38.599998  38.900002       0.007772   
2024-05-07        False   38.700001  39.090000       0.010078   
2024-05-08        False   38.669998  38.900002       0.005948   

            CumulativeReturns  
Date                           
2020-03-13          -0.040000  
2020-03-16          -0.034276